# Notes file
***
## Optimzation approach
Some methods perform a 2D to 3D or 3D to 3D optimztaion to iteratively solve the gradient problem. This works well because you can do it image by image and provide a desired skeleton model to optimize the image on. Therefore, if you are already able to estimate body size parameters this is possibly the simpliest method to arrive at a highly accurate pose estimation.  
The downside here is that the methods that perform this step, require a Jacobiean matrix. Generally this is calculated analytically and used to calculate the gradient. To use the optimization approach alone, we need to calculate this Jacobiean.

### Model fitting 3D $\Rightarrow$ 3D example
This comes from: https://link.springer.com/content/pdf/10.1007%2F978-3-540-76386-4_38.pdf, https://ieeexplore.ieee.org/document/121791   
In this case, the human body model is represented as a hierachy of joint links attached to a skin mesh.  
Optimization is done with a set of 3D data points P = {$p_1, p_2, ..., p_m$} as targets and their corresponding model vertices V = {$v_1, v_2, ...,v_m$}, the model pose vector q = {$\theta_1, \theta_2, ... \theta_n$}.
$$
\hat{q} = argmin_q ||P - V(q)||_2
$$
To solve the minimization problem, a variant of inverse kinematics is used (aka damped least square), which is inspired by the ICP algorithm. 
$$
||J \Delta q - \Delta E||^2 + \lambda||\Delta q||^2
$$
A common issue with ICP approaches for human pose tracking is that the model may drift away from the data or get stuck in local minima. An initial configuration is critical for ICP to converge correctly. The starting pose resbles a "T" shape, where the body model has its arms spread out.  
This paper does not discuss how it arrives at the Jacobiaen matrix for the body.

### Model fitting 2D $\Rightarrow$ 3D example
https://www.researchgate.net/publication/227254718_Model-Based_Pose_Estimation

#### Kinematic Chains
Human motion is articulated, to model the body effectivelty all the joints must be taken into account. For example, motion in the hand is a concatenation of motions from all the joint parents. hand <- wrist <- elbow <- shoulder <- root.  
![](kinematic_chain.png)  
In the case of a two joint arm we can represent the transformation between the spacial (root) frame and the body frame with the follwing equation:
$$
\bar{p_s} = G_{sb} \bar{p_b} = G_1 G_2 G_{sb}(0) \bar{p_b}
$$
Where Ps is the end-point position in the spacial frame, and pb is the end-point position in the body frame. $G_1$ and $G_2$ are rigid motion tranformations. $G_sb(0)$ is the transformation between B and S at the zero pose. We can extend this equation by representing the spatial coordinates of a point in the body as a function of the joint angles in the chain:
$$
\bar{p_s} = G_{sb}(\theta_1 , \theta_2) = e^{\hat{\xi_1}\theta_1} e^{\hat{\xi_2}\theta_2} G_{sb}(0) \bar{p_b} 
$$
Generalizing this procedure for any limb in the body we generate what is known as a forward kinematics map. Which defines $\Theta = (\theta_1, \theta_2, ..., \theta_n)^T$ to transform between frames S and B. 
$$
G_{sb}(\Theta) = e^{\hat{\xi_1}\theta_1} e^{\hat{\xi_2}\theta_2} ... e^{\hat{\xi_n}\theta_n} G_{sb}(0)
$$

#### The Articulated Jacobian
Used to map joint velocities to a rigid body motion. In this case we dont need it.
twist ξ = (v,ω) ∈ R6

#### Human Pose Parameterization
There are three types of DOF in the human body. Below we define the different types of joints possible. 
![](dof_table.png)  
It is a common choice to represent the body with a root joint as six free parameters, and the remaining joints as a concatenation of revolute joints. In this configuration a ball joint is simply three revolute joints. This parameterization is vunerable to sigularities. Therephore we can encode the pose configuration of a human with a scaled twist $\xi$ and a vector of n joint angles. 
$$
x_t := (\xi, \Theta), \Theta := (\theta_1, \theta_2, ..., \theta_n)
$$
In this config, a human pose is a D-dimentional state vector $x_t \in R^D$ where D = 6 + n

#### Twists
First we must define a twist. Recall that any 3D rigid motion can be represented in an exponetial form:
$$
M = exp(\theta \hat{\xi}) = exp(\begin{bmatrix} \hat{\omega} & v \\ 0_{3x1} & 0 \end{bmatrix})
$$
Consider a 3D point p rotating about an axis $\omega$ intersecting the origin with an angular velocity $\theta$. The expression $\theta \omega$ with the point p is equivalent to the cross product of the rotation axis and the point $\theta \hat{\omega} p = \theta \omega \times p$. A twist can contain six parameters and can be scaled by $\theta$ with any unit vector $\omega$. the parameter $\theta \in R$ corresponds to the motion velocity. The six twist components can be represented either as a 6D vector or as a 4×4 matrix:
$$
\eqalignno{ & \theta\xi=\theta(\omega_{1}, \omega_{2}, \omega_{3}, v_{1}, v_{2}, v_{3})\cr \theta\hat{\xi}=& \theta\left(\matrix{ 0 & -\omega_{3} & \omega_{2} & v_{1}\cr \omega_{3} & 0 & -\omega_{1} & v_{2}\cr -\omega_{2} & \omega_{1} & 0 & v_{3}\cr 0 & 0 & 0 & 0}\right)}
$$

#### Articulated Jacobian
The articulated Jacobian is a matrix $J_{\Theta} \in R^{6 \times n}$ that maps joint velocites to a rigid body motion velocity represented by a twist. Represented as:
$$
J_{\theta} = [\xi_1 \xi_2' ... \xi_n']
$$
Where $\xi_i' = (e^{\hat{\xi_1}\theta_1} ... e^{\hat{\xi_{i-1}}\theta_{i-1}}) \xi_i$ is the ith joint twist transformed to the current pose. To obtain $\xi'$ at each timestep, update the twists with the accumulated motion of parent joints in the chain. Note that the form of the Jacobian is different for every limb since different body parts are influenced by different joints.  
Given a pose determined by $\Theta$ and point in the body in spacial coordinates $p_s$ we can obtain $\Delta p_s$ in position as a function of the increment in the parameter space $\Delta \Theta$ as:
$$
\Delta \bar{p_s} = [J_{\Theta} * \Delta \Theta]^{\wedge} \bar{p_s} = [\xi_1 \Delta \theta_1 + \xi_2' \Delta \theta_2 + ... + \xi_n' \Delta \theta_m]^{\wedge}\bar{p_s}
$$

#### The Pose Jacobian
To optimize the pose parameterizations we need to know the relationships between the increments in pose parameters and increments in the position of a point of a body segment. This is given by the pose Jacobian $J_x(p_s) = \frac{\Delta p_s}{\Delta x}$. Given the expression above for $\Delta \bar{p_s}$ we can denote $\Delta \xi = [\Delta v_1, \Delta v_2, \Delta v_3, \Delta \omega_1, \Delta \omega_2, \Delta \omega_3] $ as the relative twist corresponding to the root joint. The six coordinates of the scaled relative twist $\Delta \xi$ are now free parameters to estimate. Rewrite the above equation using the identity $[u+x]^{\wedge} = \hat{u} + \hat{w}$.  

$$
\Delta p_s = [\Delta \xi + \xi_1' \Delta \theta_1 + ... + \xi_n' \Delta \theta_n]^{\wedge}\bar{p_s} \\
\Delta p_s = \hat{\Delta \xi} \bar{p_s} + \hat{\Delta \xi_1'} \bar{p_s} \Delta \theta_1 + ... + \hat{\Delta \xi_n'} \bar{p_s} \Delta \theta_n
$$
We can isolate the parameters of the root joint $\Delta \xi$ rewriting as $\hat{\Delta \xi} \bar{p_s}$
$$
\hat{\Delta \xi} \bar{p_s} = \Delta v + \Delta \omega \times p_s = \Delta v - p_s^{\wedge} \Delta \omega = [I_{[3 \times 3]} | -p_s^{\wedge}] \Delta \xi
$$
Sub these expressions together and achieve:
$$
\Delta p_s = [I_{[3 \times 3]} | -p_s^{\wedge}] \Delta \xi + \hat{\Delta \xi} \bar{p_s} + \hat{\Delta \xi_1'} \bar{p_s} \Delta \theta_1 + ... + \hat{\Delta \xi_n'} \bar{p_s} \Delta \theta_n \\
\Delta p_s = J_x(p_s) \Delta x
$$
where $\Delta x = [\Delta \xi \Delta \Theta]$ is the differential vector of pose parameters and
$$
J_x(p_s) = [I_{[3 \times 3]}  -p_{\hat{s}} \hat{\xi_1}\bar{p_s} \hat{\xi_2'}\bar{p_s} ... \hat{\xi_n'}\bar{p_s}]
$$
is the position Jacobian $J_x(p_s) \in R^{3 \times D}$ of a point $p_s$ with respect to the pose parameters which we denote as pose Jacobian.  
For a given point in the body $p_s$ in a configuration x, $J_x(p_s) : R^D -> R^3$ maps an increment of the pose parameters $\Delta x$ to a positional increment of the point $\Delta p_s$  
***
There are two main blocks to the pose Jacobian. The first 6 column vectors correspond to the non constraint relative twist $\Delta \xi$ of the root joint, and the rest of the columns (joint columns) that correspond to the point velocity contribution of each angle. The column entries of joints that are not parents of the point are simply $0_{3 \times 1}$.

## Building a toy Jacobian
![](kinematic_chain.png)

In [281]:
# fig above:
# Blue is Z, Green is Y, Red is X

import numpy as np
# defining the static model parameters
L1 = 1.0 # m
L2 = 1.5 # m

In [38]:
def skew(vector):
    """
    this function returns a numpy array with the skew symmetric cross product matrix for vector.
    the skew symmetric cross product matrix is defined such that
    np.cross(a, b) = np.dot(skew(a), b)

    :param vector: An array like vector to create the skew symmetric cross product matrix for
    :return: A numpy array of the skew symmetric cross product vector
    """

    return np.array([[0, -vector[2], vector[1]], 
                     [vector[2], 0, -vector[0]], 
                     [-vector[1], vector[0], 0]])

In [296]:
def rodriguez_formula(theta, w_hat):
    return np.eye(3) + w_hat * np.sin(theta) + np.power(w_hat, 2)*(1-np.cos(theta))

def calc_G(w, v, theta):
    # eq 9.18
    G = np.zeros((4,4))
    G[:3, :3] = rodriguez_formula(theta, skew(w))
    t = (np.eye(3) - G[:3, :3])@(np.cross(w,v)) #+ w@w.T * v * theta
    G[:3, 3] = t
    G[3, 3] = 1
    return G

In [297]:
def get_ps(theta_1, theta_2, pb=np.array([0, 0, 0, 1])):
    # coordiantes of a control point in Ps given Pb
    # eq 9.27    
    Gsb_0 = np.array([[1, 0, 0, 0],
                      [0, 1, 0, L1],
                      [0, 0, 1, 0],
                      [0, 0, 0, 1]])

    # w is a unit vector specifing the direction of the rotation
    # w_hat is the skew matrix calculated from w
    w_1 = np.array([1, 0, 0])
    w_2 = np.array([1, 0, 0])
    # v is the translation along the axis of rotation. 
    v_1 = np.cross(-w,np.array([0, 0, 0]))
    v_2 = np.cross(-w,np.array([0, 0, 0]))
    # G is the rigid motion written in exponetial form
    G_1 = calc_G(w_1, v_1, theta_1)
    G_2 = calc_G(w_2, v_2, theta_2)
    print(G_1)
    print(G_2)
    # theta is the angle of rotation in radians
    # R = exp(theta w_hat)
    return G_1 @ Gsb_0 @ pb
    



In [298]:
get_ps(0, 0)

[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


array([0., 1., 0., 1.])

In [299]:
n = get_ps(np.pi / 3, 0)
n[0:3]

[[ 1.         0.         0.         0.       ]
 [ 0.         1.        -0.3660254  0.       ]
 [ 0.         1.3660254  1.         0.       ]
 [ 0.         0.         0.         1.       ]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


array([0.       , 1.       , 1.3660254])

In [295]:
np.linalg.norm(n[0:3])

1.6929339632083815

In [276]:
w = np.array([0, 0, 1])
v = np.array([L1, 0, 0])
theta = np.pi / 3
p1 = np.eye(3) - rodriguez_formula(theta, skew(w))
p2 = np.cross(w,v)
p3 = w*w.T*v*theta

print(p1)
print(p2)
print(p3)
print(p1 * p2 + p3)
print(np.linalg.norm(p1 * p2 + p3))


[[ 0.         0.3660254  0.       ]
 [-1.3660254  0.         0.       ]
 [ 0.         0.         0.       ]]
[0. 1. 0.]
[0. 0. 0.]
[[0.        0.3660254 0.       ]
 [0.        0.        0.       ]
 [0.        0.        0.       ]]
0.3660254037844387


In [255]:
 w.T@w * v 

array([1., 0., 0.])

In [277]:
np.cross(-w,np.array([0, 0, 0]))

array([0, 0, 0])

array([1.04719755, 0.        , 0.        ])